In [1]:
import numpy as np
import pandas as pd


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('/content/covid_toy.csv')


In [5]:
df.head(4)


,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No


In [7]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [8]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [11]:
X_train

,age,gender,fever,cough,city
33,26,Female,98.0,Mild,Kolkata
18,64,Female,98.0,Mild,Bangalore
97,20,Female,101.0,Mild,Bangalore
56,71,Male,NaN,Strong,Kolkata
65,69,Female,102.0,Mild,Bangalore
...,...,...,...,...,...
71,75,Female,104.0,Strong,Delhi
52,47,Female,100.0,Strong,Bangalore
98,5,Female,98.0,Strong,Mumbai
47,18,Female,104.0,Mild,Bangalore


In [12]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])


In [13]:
X_train_fever

array([[ 98.        ],
       [ 98.        ],
       [101.        ],
       [100.89041096],
       [102.        ],
       [104.        ],
       [104.        ],
       [104.        ],
       [ 99.        ],
       [101.        ],
       [100.        ],
       [ 99.        ],
       [100.89041096],
       [101.        ],
       [101.        ],
       [ 99.        ],
       [ 99.        ],
       [102.        ],
       [103.        ],
       [104.        ],
       [102.        ],
       [100.        ],
       [100.        ],
       [100.        ],
       [ 98.        ],
       [101.        ],
       [102.        ],
       [100.        ],
       [102.        ],
       [ 98.        ],
       [ 98.        ],
       [102.        ],
       [102.        ],
       [100.89041096],
       [101.        ],
       [103.        ],
       [ 98.        ],
       [103.        ],
       [104.        ],
       [ 98.        ],
       [ 98.        ],
       [104.        ],
       [100.        ],
       [101

In [14]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [15]:
X_test_cough

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.]])

In [16]:
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [18]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)
X_train_transformed.shape

(80, 7)

In [19]:
X_train_transformed

array([[ 26.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 64.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 20.        , 101.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 71.        , 100.89041096,   1.        ,   0.        ,
          1.        ,   0.        ,   1.        ],
       [ 69.        , 102.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 51.        , 104.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 17.        , 104.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 12.        , 104.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 65.        ,  99.        ,   1.        ,   1.        ,
          0.    

In [21]:
from sklearn.compose import ColumnTransformer

In [22]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [28]:
transformer.fit_transform(X_train)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  20.        ],
       [100.89041096,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [102.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  69.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  51.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [ 99.        ,   0.        ,   1.        ,   1.        ,
          0.    

In [24]:
X_train_new = transformer.fit_transform(X_train).shape


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
X_train_new

(80, 7)